In [1]:
from camelyonpatch import CamelyOnPatch
import tensorflow as tf
import os
import numpy as np
os.environ['CUDA_VISIBLE_DEVICES']='0'


In [2]:
cpd=CamelyOnPatch('/home/jsearcy/Desktop/ML Data Sets/pcamv1')
print(cpd.x_train[0])



[[[0.88627451 0.81960784 0.88235294]
  [0.87843137 0.82352941 0.88235294]
  [0.8745098  0.82745098 0.8745098 ]
  ...
  [0.85490196 0.69803922 0.90196078]
  [0.83921569 0.71372549 0.86666667]
  [0.7372549  0.63137255 0.75294118]]

 [[0.8745098  0.81960784 0.87843137]
  [0.87058824 0.82352941 0.87843137]
  [0.86666667 0.82745098 0.87058824]
  ...
  [0.49803922 0.34117647 0.58823529]
  [0.62352941 0.49411765 0.70980392]
  [0.8        0.69019608 0.88235294]]

 [[0.87058824 0.82352941 0.87843137]
  [0.86666667 0.82352941 0.87843137]
  [0.85882353 0.82745098 0.87058824]
  ...
  [0.30196078 0.14901961 0.45882353]
  [0.16470588 0.03137255 0.32941176]
  [0.5372549  0.41960784 0.70196078]]

 ...

 [[0.48235294 0.31764706 0.64705882]
  [0.50196078 0.35294118 0.67843137]
  [0.49411765 0.35686275 0.67843137]
  ...
  [0.79607843 0.52941176 0.82352941]
  [0.88627451 0.60784314 0.8745098 ]
  [0.79215686 0.50980392 0.76078431]]

 [[0.35294118 0.20392157 0.54509804]
  [0.35294118 0.21960784 0.55686275]


In [3]:
def add_target_area(x):
    batch=tf.shape(x)[0]
    
    target_area=np.zeros((1,96,96,3),dtype='float32')
    target_area[0,32:64,32:64,:]=1
    tf_area=tf.constant(target_area)
    tf_area=tf.tile(tf_area,[batch,1,1,1])
    output=tf.concat([x,tf_area],axis=-1)
    return output


print(dir(tf.keras.applications))
image_input=tf.keras.layers.Input(shape=(96,96,3))
cnn=tf.keras.layers.Lambda(add_target_area)(image_input)

small_feature=tf.keras.layers.Conv2D(64,10,padding='same')(cnn)
small_output=tf.keras.layers.GlobalAveragePooling2D()(small_feature)


cnn1=tf.keras.layers.Conv2D(64,4,padding='same')(cnn)
cnn=tf.keras.layers.LeakyReLU()(cnn1)

cnn=tf.keras.layers.Conv2D(64,4,padding='same')(cnn)
cnn=tf.keras.layers.LeakyReLU()(cnn)
cnn=tf.keras.layers.Conv2D(64,4,padding='same')(cnn)
cnn=tf.keras.layers.LeakyReLU()(cnn)
cnn=tf.keras.layers.Add()([cnn,cnn1])
cnn=tf.keras.layers.Dropout(.3)(cnn)

cnn2=tf.keras.layers.Conv2D(128,4,2,padding='same')(cnn1)
cnn=tf.keras.layers.LeakyReLU()(cnn2)

cnn=tf.keras.layers.Conv2D(128,4,padding='same')(cnn)
cnn=tf.keras.layers.LeakyReLU()(cnn)
cnn=tf.keras.layers.Conv2D(128,4,padding='same')(cnn)
cnn=tf.keras.layers.LeakyReLU()(cnn)
cnn=tf.keras.layers.Add()([cnn,cnn2])
cnn=tf.keras.layers.Dropout(.3)(cnn)



cnn=tf.keras.layers.GlobalAveragePooling2D()(cnn)
cnn=tf.keras.layers.Concatenate()([cnn,small_output])
cnn=tf.keras.layers.Dropout(.5)(cnn)
cnn=tf.keras.layers.Dense(100)(cnn)
cnn=tf.keras.layers.Dropout(.5)(cnn)
cnn=tf.keras.layers.Dense(1,activation='sigmoid')(cnn)


model=tf.keras.models.Model(image_input,cnn)

model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

model.summary()



#pretrained_model=tf.keras.applications.VGG19(include_top=False,weights='imagenet',input_shape=(96,96,3))





['DenseNet121', 'DenseNet169', 'DenseNet201', 'InceptionResNetV2', 'InceptionV3', 'MobileNet', 'MobileNetV2', 'NASNetLarge', 'NASNetMobile', 'ResNet50', 'VGG16', 'VGG19', 'Xception', '__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__path__', '__spec__', 'densenet', 'inception_resnet_v2', 'inception_v3', 'mobilenet', 'mobilenet_v2', 'nasnet', 'resnet50', 'vgg16', 'vgg19', 'xception']
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 96, 96, 3)    0                                            
__________________________________________________________________________________________________
lambda (Lambda)                 (None, 96, 96, 6)    0           input_1[0][0]                    
________________________________________________________________________

In [4]:
print(np.max(cpd.x_develop))

1.0


In [5]:
batch_size=100
es=tf.keras.callbacks.EarlyStopping(patience=10)

model.fit_generator( 
                    cpd.datagen.flow(cpd.x_train,np.squeeze(cpd.y_train),batch_size=batch_size ),
                    steps_per_epoch=300,
                    validation_data=(cpd.x_develop,np.squeeze(cpd.y_develop)),                    
                    epochs=100,callbacks=[es])


Epoch 1/100
300/300 [==============================] - 68s 227ms/step - loss: 0.6258 - acc: 0.6600 - val_loss: 0.5944 - val_acc: 0.6781
Epoch 2/100
300/300 [==============================] - 63s 210ms/step - loss: 0.5264 - acc: 0.7511 - val_loss: 0.5719 - val_acc: 0.7209
Epoch 3/100
300/300 [==============================] - 63s 211ms/step - loss: 0.5099 - acc: 0.7637 - val_loss: 0.5535 - val_acc: 0.7238
Epoch 4/100
300/300 [==============================] - 63s 209ms/step - loss: 0.4899 - acc: 0.7750 - val_loss: 0.5855 - val_acc: 0.6894
Epoch 5/100
300/300 [==============================] - 63s 210ms/step - loss: 0.4761 - acc: 0.7796 - val_loss: 0.6153 - val_acc: 0.6568
Epoch 6/100
300/300 [==============================] - 63s 210ms/step - loss: 0.4605 - acc: 0.7906 - val_loss: 0.5516 - val_acc: 0.6802
Epoch 7/100
300/300 [==============================] - 63s 210ms/step - loss: 0.4647 - acc: 0.7887 - val_loss: 0.5081 - val_acc: 0.7406
Epoch 8/100
300/300 [===========================

In [6]:
import matplotlib.pyplot as plt

y_pred_train=np.squeeze(model.predict(cpd.x_train[0:10000]))
plt.hist([v for v,p in zip(y_pred_train,np.squeeze(cpd.y_train[0:10000])) if p ],range=(0,1),bins=50,density=True,histtype='step')
plt.hist([v for v,p in zip(y_pred_train,np.squeeze(cpd.y_train[0:10000])) if not p ],range=(0,1),bins=50,density=True,histtype='step')
plt.show()



y_pred_test=np.squeeze(model.predict(cpd.x_test))
plt.hist([v for v,p in zip(y_pred_test,np.squeeze(cpd.y_test)) if p ],range=(0,1),bins=50,density=True,histtype='step')
plt.hist([v for v,p in zip(y_pred_test,np.squeeze(cpd.y_test)) if not p ],range=(0,1),bins=50,density=True,histtype='step')
plt.show()

y_pred_develop=np.squeeze(model.predict(cpd.x_develop))
plt.hist([v for v,p in zip(y_pred_develop,np.squeeze(cpd.y_develop)) if p ],range=(0,1),bins=50,density=True,histtype='step')
plt.hist([v for v,p in zip(y_pred_develop,np.squeeze(cpd.y_develop)) if not p ],range=(0,1),bins=50,density=True,histtype='step')
plt.show()


<Figure size 640x480 with 1 Axes>

<Figure size 640x480 with 1 Axes>

<Figure size 640x480 with 1 Axes>

In [31]:
import openslide
import matplotlib.pyplot as plt
from zipfile import ZipFile
import xml.etree.ElementTree as ET

#tiff_file='/home/jsearcy/Desktop/ML Data Sets/pcamv1/tumor_001.tif'
tiff_file='/home/jsearcy/Desktop/ML Data Sets/pcamv1/normal_001.tif'

root = ET.fromstring(ZipFile("/home/jsearcy/Desktop/ML Data Sets/lesion_annotations_train.zip").open('tumor_001.xml').read())
 




slide_image=openslide.OpenSlide(tiff_file)
coords=slide_image.level_dimensions[0]
res=slide_image.level_dimensions[2]


In [32]:
len(root.getchildren())
tumor_coords=[(float(i.attrib['X']),float(i.attrib['Y'])) for i in root.iter() if i.tag=="Coordinate"]

In [33]:
x_offset=35000
y_offset=100000
image=np.asarray(slide_image.read_region( (x_offset,y_offset),2,(800*16,800*16)  ))


    



In [34]:
new_tumor=[]
for x,y in tumor_coords:
    new_x=(x-x_offset)/4.
    new_y=(y-y_offset)/4.
    new_tumor.append((new_x,new_y))
    
new_tumor_small=[]
for x,y in tumor_coords:
    new_x=(x-x_offset)/(4.*32.)
    new_y=(y-y_offset)/(4.*32.)
    new_tumor_small.append((new_x,new_y))
    
    

In [35]:
new_x,new_y

(283.28359375, 216.9609375)

In [36]:
%matplotlib qt
f=plt.figure(figsize=(20,40))
plt.imshow(image)
plt.scatter(*zip(*new_tumor))

In [37]:
res_x,res_y,chann=image.shape
new_image=np.zeros((res_x//32,res_y//32,3))
mask=np.zeros((res_x//32,res_y//32))
        
for x in range(0,res_x-96,32):
    pred_data=[]
    for y in range(0,res_y-96,32):
        data=image[x:x+96,y:y+96,:]
        pred_data.append(np.expand_dims(data[:,:,0:3],0)/255.)                
        color=np.mean(np.mean(data[32:64,32:64,:],axis=0),axis=0)
#                print(color)
#                print(x//32,y//32)
        new_image[x//32,y//32,:]=color[0:3]
    output=model.predict(np.concatenate(pred_data,axis=0))
    mask[x//32,0:len(output)]= np.squeeze(output)
                



In [38]:
coords,coords[0]*coords[1]

((97792, 221184), 21630025728)

In [39]:
#f=plt.figure(figsize=(10,20))

#plt.imshow(new_image/255.)
#plt.scatter(*zip(*new_tumor_small))
#plt.show()

f=plt.figure(figsize=(10,20))
plt.imshow(mask>0.997,cmap='Reds')
plt.scatter(*zip(*new_tumor_small),s=.2)

plt.show()


In [15]:
(image[:,:,3]==255).all()


True

In [16]:
image=tf.keras.layers.Lambda(lambda x: tf.image.resize_image_with_crop_or_pad(
    x*255,
    96,
    96
))(image_input)


In [17]:
pretrained_model=tf.keras.applications.ResNet50(include_top=False,weights='imagenet',input_shape=(96,96,3),input_tensor=image)

pretrained_output=pretrained_model.output
pretrained_input=pretrained_model.input

for layer in pretrained_model.layers:
    layer.trainable=False


/home/jsearcy/anaconda3/envs/tensorflow/lib/python3.6/site-packages/keras_applications/resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


In [18]:
#flatten=tf.keras.layers.GlobalMaxPooling2D()(pretrained_output)
flatten=tf.keras.layers.Flatten()(pretrained_output)
top=tf.keras.layers.Dropout(0.9)(flatten)
top=tf.keras.layers.Dense(500)(top)
top=tf.keras.layers.LeakyReLU()(top)
top=tf.keras.layers.Dropout(0.6)(top)

top=tf.keras.layers.Dense(500)(top)
top=tf.keras.layers.LeakyReLU()(top)
top=tf.keras.layers.Dropout(0.6)(top)

last_layer=tf.keras.layers.Dense(1,activation='sigmoid')(top)
res_model=tf.keras.models.Model(pretrained_input,last_layer)

res_model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

res_model.summary()


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 96, 96, 3)    0                                            
__________________________________________________________________________________________________
lambda_1 (Lambda)               (None, 96, 96, 3)    0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 102, 102, 3)  0           lambda_1[0][0]                   
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 48, 48, 64)   9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (

In [19]:
batch_size=100
es=tf.keras.callbacks.EarlyStopping(patience=3)

res_model.fit_generator( 
                    cpd.datagen.flow(cpd.x_train,np.squeeze(cpd.y_train),batch_size=batch_size ),
                    steps_per_epoch=300,
                    validation_data=(cpd.x_develop,np.squeeze(cpd.y_develop)),                    
                    epochs=100,
                    callbacks=[es])

Epoch 1/100
300/300 [==============================] - 63s 210ms/step - loss: 2.1478 - acc: 0.6937 - val_loss: 0.5558 - val_acc: 0.7183
Epoch 2/100
300/300 [==============================] - 60s 198ms/step - loss: 0.6037 - acc: 0.7354 - val_loss: 0.5851 - val_acc: 0.7064
Epoch 3/100
300/300 [==============================] - 60s 199ms/step - loss: 0.5034 - acc: 0.7645 - val_loss: 0.5656 - val_acc: 0.7191
Epoch 4/100
300/300 [==============================] - 60s 199ms/step - loss: 0.4908 - acc: 0.7712 - val_loss: 0.5649 - val_acc: 0.7079


In [20]:
import matplotlib.pyplot as plt

y_pred_train=np.squeeze(res_model.predict(cpd.x_train[0:10000]))
plt.hist([v for v,p in zip(y_pred_train,np.squeeze(cpd.y_train[0:10000])) if p ],range=(0,1),bins=50,density=True,histtype='step')
plt.hist([v for v,p in zip(y_pred_train,np.squeeze(cpd.y_train[0:10000])) if not p ],range=(0,1),bins=50,density=True,histtype='step')
plt.show()



y_pred_test=np.squeeze(res_model.predict(cpd.x_test))
plt.hist([v for v,p in zip(y_pred_test,np.squeeze(cpd.y_test)) if p ],range=(0,1),bins=50,density=True,histtype='step')
plt.hist([v for v,p in zip(y_pred_test,np.squeeze(cpd.y_test)) if not p ],range=(0,1),bins=50,density=True,histtype='step')
plt.show()

y_pred_develop=np.squeeze(res_model.predict(cpd.x_develop))
plt.hist([v for v,p in zip(y_pred_develop,np.squeeze(cpd.y_develop)) if p ],range=(0,1),bins=50,density=True,histtype='step')
plt.hist([v for v,p in zip(y_pred_develop,np.squeeze(cpd.y_develop)) if not p ],range=(0,1),bins=50,density=True,histtype='step')
plt.show()

In [21]:
postive=[v for v in np.argsort(y_pred_develop) if cpd.y_develop[v]]
negative=[v for v in np.argsort(y_pred_develop) if not cpd.y_develop[v]]

f=plt.figure(figsize=(20,2))
plt.imshow(np.hstack(cpd.x_develop[postive[0:10]]))
plt.show()
f=plt.figure(figsize=(20,2))

plt.imshow(np.hstack(cpd.x_develop[negative[10:20]]))
plt.show()
f=plt.figure(figsize=(20,2))

plt.imshow(np.hstack(cpd.x_develop[postive[20:30]]))

f=plt.figure(figsize=(10,10))
plt.imshow(cpd.x_develop[postive[20]] )
print(y_pred_develop[postive[9]])
plt.show()






TypeError: Indexing elements must be in increasing order

In [ ]:
#model.fit(cpd.x_train,np.squeeze(cpd.y_train),
#          validation_data=(cpd.x_develop,np.squeeze(cpd.y_develop)),                    
#          epochs=10,shuffle='batch')

In [ ]:
sort=np.argsort(y_pred)
#print(cpd.y_test[sort[0:20]])
index=8
print(cpd.y_test[sort[index]],y_pred[sort[index]])
plt.imshow(cpd.x_test[sort[index]])







In [ ]:
sess=tf.keras.backend.get_session()
im_out=sess.run(image,feed_dict={image_input:cpd.x_test[0:10]})

plt.imshow(cpd.x_test[3])
plt.show()
print(np.min(im_out),np.max(im_out))
plt.imshow(im_out[3].astype('uint8'))

plt.show()


In [ ]:
import matplotlib.pyplot as plt
f=plt.figure(figsize=(30,5))
plt.imshow(np.hstack(cpd.x_test[0:10]))
plt.show()
f=plt.figure(figsize=(30,5))

plt.imshow(np.hstack(cpd.x_train[0:10]))
plt.show()
f=plt.figure(figsize=(30,5))

plt.imshow(np.hstack(cpd.x_develop[0:10]))

plt.show()

In [ ]:
cpd.x_test[1]/255.
